In [139]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import os

In [140]:
# Function to extract Product Title
def get_title(soup):

    # Outer Tag Object
    title = soup.find("h1", attrs={"class":'title-product'})
    
    # Inner NavigatableString Object
    title_value = title.text

    # Title as a string value
    title_string = title_value.strip().split('\n')
    
    if title_string == ['']:
        return 'Không xác định'
    return title_string[0]

# Function to extract Product Price
def get_info(soup):

    array_1 = soup.find_all("ul", attrs={'class':'list-attr-hot clearfix'})[0].text.strip().split('\n')
    array_2 = soup.find_all("ul", attrs={'class':'list-attr-hot clearfix'})[1].text.strip().split('\n')
    
    if(array_2[0] != ''):
        filtered_array_2 = [item for item in array_2 if item]
    else: filtered_array_2 = []
    
    filtered_array_1 = [item for item in array_1 if item]
    
    
    if len(filtered_array_2) == 2:
        return [filtered_array_1[1], filtered_array_1[3], filtered_array_1[5], filtered_array_2[1], 0, 0]
    elif len(filtered_array_2) == 4:
        return [filtered_array_1[1], filtered_array_1[3], filtered_array_1[5], filtered_array_2[1], filtered_array_2[3], 0]
    elif len(filtered_array_2) == 6:
        return [filtered_array_1[1], filtered_array_1[3], filtered_array_1[5], filtered_array_2[1], filtered_array_2[3], filtered_array_2[5]]

    return [filtered_array_1[1], filtered_array_1[3], filtered_array_1[5], 0, 0, 0]
# Function to extract Product Rating
def get_location(soup):

    location = soup.find("ul", attrs={'class':'breadcrumb clearfix'}).text.strip().split('\n')

    if location != ['']:
        return [location[1], location[0]]
    return ['Không xác định', 'Không xác định']

# Function to extract Number of User Reviews
def get_illustrate(soup):

    array = soup.find("div", attrs={'class':'ct-pr-sum'}).text.strip().split('\n')
    if 'Tìm kiếm theo từ khóa:' in array:
        index = array.index('Tìm kiếm theo từ khóa:')
        array = array[:index]

    return ' '.join(array)

# Function to extract Availability Status
def get_image(soup, id):
    
    img_url = soup.find("img", attrs={'class':'img-product-face'})
    if(img_url):
        img_url = 'https://bds.com.vn/images/noimage.png'
    else:
        img_url = soup.find_all("img")[1]['src']
        
    
    idx = img_url.rfind('.')
    img_name = f'img{id}' + img_url[idx:]
    img_response = requests.get(img_url, stream=True)   
    
        
    with open('img/'+img_name, 'wb') as f:
        f.write(img_response.content)
        
    return img_name



In [141]:
id = 1
data = []
pages = 200

In [142]:

if __name__ == '__main__':


    # The webpage URL
    URLs = ['https://bds.com.vn/mua-ban-nha-dat', 'https://bds.com.vn/mua-ban-dat', 'https://bds.com.vn/cho-thue-nha-dat']

    for idx, URL in enumerate(URLs):
        
        for i in range(1, pages):
            # Fetch links as List of Tag Objects
            # HTTP Request
            webpage = requests.get(URL)

            # Soup Object containing all data
            soup = BeautifulSoup(webpage.content, "html.parser")
            
            links = soup.find_all("a", attrs={'class':'title-item-nhadat'})

            # Store the links
            links_list = []

            # Loop for extracting links from Tag Objects
            for link in links:
                    links_list.append(link.get('href'))

            
            # Loop for extracting product details from each link 
            for link in links_list:
                print(link)
                d = {
                    "_id": id,
                    "title": str,
                    "location": str,
                    "price": str,
                    "area": str,
                    "date": str,
                    "image": str,
                    "illustrate": str,
                    "types": str,
                    "option": str,
                    "num_bedroom": 0,
                    "num_wc": 0,
                    "num_floor": 0,
                }
                new_webpage = requests.get(link)

                new_soup = BeautifulSoup(new_webpage.content, "html.parser")
                
                info = get_info(new_soup)
                location = get_location(new_soup)
                
                if idx == 0:
                    d['types'] = 'Mua bán nhà đất'
                elif idx == 1:
                    d['types'] = 'Mua bán đất'
                else:
                    d['types'] = 'Cho thuê nhà đất'

                # Function calls to display all necessary product information
                d['title'] = get_title(new_soup)
                d['location'] = location[1]
                d['price'], d['area'], d['date'] = info[0], info[1], info[2]
                d['image'] = get_image(new_soup, id)
                d['illustrate'] = get_illustrate(new_soup)
                d['option'] = location[0]
                d['num_bedroom'] = info[3]
                d['num_wc'] = info[4]
                d['num_floor'] = info[5]
                
                data.append(d)
                id += 1
            URL = URLs[idx] + f'-page{i}'



https://bds.com.vn/chinh-chu-ban-dat-duong-do-nhuan-quan-tan-phu-sat-ben-aeon-mall-tan-phu-p634836.html
https://bds.com.vn/chinh-chu-ban-gap-dat-tang-nha-cach-bien-20-buoc-chan-khu-vuc-yen-tinh-p612865.html
https://bds.com.vn/ban-nha-chinh-chu-hem-373-ly-thuong-kiet-p633045.html
https://bds.com.vn/can-ho-chuan-nhat-ban-tai-hai-phong-cach-aeon-mall-500m-p634436.html
https://bds.com.vn/ban-nha-ngo-82-tho-nhuom-6-tang-ngo-rong-nha-moi-xay-thang-may-da-cong-nang-p636180.html
https://bds.com.vn/ban-nha-cap-4-mat-tien-pham-the-hien---phuong-thang-tam---tp-vung-tau---gia-tot-6-ty-6-p636456.html
https://bds.com.vn/du-an-to-hop-van-phong-va-can-ho-handico-33-le-van-luong-p636340.html
https://bds.com.vn/-p635386.html
https://bds.com.vn/gd-can-tien-ban-nhanh-nha-thuong-thanh-40m2-3-ngu-465-ty-p636505.html
https://bds.com.vn/sieu-pham-xuan-la---ngoai-giao-doan---cong-nang-day-du-p636468.html
https://bds.com.vn/mua-ban-nha-huyen-binh-chanh-gia-re-moi-nhat--vi-tri-dac-dia-p636466.html
https://bds.co

IndexError: list index out of range

In [138]:
import json
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
from PIL import Image
import os

# Thư mục chứa các ảnh gốc
input_dir = 'img/'

# Thư mục để lưu các ảnh đã thay đổi kích thước
output_dir = 'resized_images/'

# Tạo thư mục đầu ra nếu chưa tồn tại
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Lặp qua tất cả các tệp trong thư mục ảnh gốc
for img_file in os.listdir(input_dir):
    if img_file.endswith(('jpg', 'jpeg', 'png', 'bmp', 'gif')):  # Kiểm tra định dạng file ảnh
        # Đường dẫn đầy đủ đến file ảnh
        img_path = os.path.join(input_dir, img_file)
        
        # Mở ảnh
        img = Image.open(img_path)

        # Thay đổi kích thước ảnh về 200x250
        img_resized = img.resize((250, 250))

        # Lưu ảnh đã thay đổi kích thước vào thư mục đầu ra
        output_path = os.path.join(output_dir, img_file)
        img_resized.save(output_path)

        # print(f'Đã thay đổi kích thước: {img_file}')
